In [ ]:
import os
from pyrosm import OSM
import osmnx as ox
import geopandas as gpd
import pandas as pd
import uuid
import matplotlib.pyplot as plt

#project specific packages
import services.network_bands as network_bands
import services.batch_csv as batch_csv
import services.census as census

In [ ]:
#https://build.nisra.gov.uk/en/custom/data?d=HOUSEHOLD&v=DZ21&v=HH_LIFESTAGE_AGG15

In [ ]:
#set base directory for data file paths.
base_dir = os.getcwd()

In [ ]:
# create network graph and edges.
base_road_path = f'{base_dir}\\testEnvironment\\Data\\belfast_slightly_trimmed.osm.pbf'
G, nodes, edges = network_bands.load_osm_network(file_path=base_road_path, network_type='driving', graph_type='networkx')

In [ ]:
#Start locations
start_locations = pd.read_csv(f'{base_dir}\\testEnvironment\\Data\\libraries_belfast_2024.csv')
print(start_locations.columns)
#Ensure data is converted to a dataframe
start_locations_gdf = network_bands.csv_to_gdf(start_locations, 'X COORDINATE', 'Y COORDINATE', 29902, 4326)
print(start_locations_gdf)

In [ ]:
#Create the network areas and contours - Considering making this into a Class with basic GUI, but for now fine as this.
start_locations_nearest_node = network_bands.nearest_node_and_name(G, start_locations=start_locations_gdf,  
                                                                   location_name = 'Static Library Name')

#Create service areas for each distance. Remember, these overlap, good for counting checking how close 
#input custom distances as a list.
search_distances = [1000,2000,3000]
#Create individual network areas around start locations. alpha area of 500 quite good for Belfast
network_areas = network_bands.network_areas(nearest_node_dict=start_locations_nearest_node, graph=G, 
                                                    search_distances=search_distances, alpha_value=500, weight = 'length', progress=True)
#Create network area contours by dissolving and differencing polygons based on distance.
network_contours = network_bands.network_contours(network_areas, dissolve_cat='distance',aggfunc='first', show_graph=True)

In [ ]:
#Load in data zones from 2021 census
#Ensure evrything's in 4326 for network analysis, probably can change it back to tm65.
data_zones = gpd.read_file(f'{base_dir}\\testEnvironment\\Data\\DZ2021.shp')
data_zones.to_crs(4326, inplace=True)
#extract only belfast datazones
belfast_zones = data_zones[data_zones['LGD2014_nm'] == 'Belfast']
#Load in house data 
pointer = gpd.read_file(f'{base_dir}\\testEnvironment\\Data\\pointer_randomised.shp')
pointer['uuid'] = pointer.apply(lambda _: uuid.uuid4(), axis=1)
pointer.to_crs(4326, inplace=True)

In [ ]:
# fig, ax = plt.subplots(figsize=(12,12))

# edges.plot(ax=ax, zorder=2)
# pointer.plot(ax=ax, color='black', zorder = 12, markersize=2)
# start_locations_gdf.plot(ax=ax, color='red', zorder=13, markersize=150)
# belfast_zones.plot(ax=ax, color='gray', zorder=11)


In [ ]:
#Performa spatial join of pointer households and datazones in belfast
joined_gdf = gpd.sjoin(pointer, belfast_zones, how='inner', op='within')

# Count the points per data zone using 'index_right'  which = belfast_zones
point_counts = joined_gdf.groupby('index_right').size()
point_counts = point_counts.reset_index(name='actual_households')

# Merge the counts back to the belfast data zones GeoDataFrame, right_on =right as datazones are the right joined gdf.
belfast_zones = belfast_zones.merge(point_counts, how='left', left_index=True, right_on='index_right')
# Turn NaN to 0
belfast_zones['actual_households'] = belfast_zones['actual_households'].fillna(0).reset_index(drop=True)

In [ ]:
#Load the Census data, file_paths in file_paths.
file_paths = [
    '/testEnvironment/Data/census_data/ni-2021-usual-residents.csv',
    '/testEnvironment/Data/census_data/ni-2021-households.csv',
    '/testEnvironment/Data/census_data/ni-2021-employment-deprivation.csv'
]
#extract each one from dataframe
loaded_csv = batch_csv.batch_csv_read(file_paths)

#check data is loaded loaded
print(loaded_csv.keys())

#force rename to maintain consistency of important join value column.
loaded_csv['ni-2021-employment-deprivation'].rename(columns={'Census 2021 Data Zone Code':'Geography code'}, inplace=True)

#OSNI data has irregular capitalisation, avoids this bug by forcing lower case. Some are 'Geography Code', 'geography Code' etc.
for key, df in loaded_csv.items():
    df.columns = df.columns.str.lower()
    
joined_census_data = census.join_census_csv(loaded_csv, 'geography code',  drop=True,join_type='left')

In [ ]:
#Calculate CENSUS METRICS PER HOUSE in pointer data
#join the census data loaded 
belfast_zones = pd.merge(joined_census_data, belfast_zones, left_on='geography code', right_on='DZ2021_cd')
#Need to force these to numeric. Ensure coerce for any nulls
belfast_zones['all households'] = pd.to_numeric(belfast_zones['all households'], errors = 'coerce')
belfast_zones['all usual residents'] = pd.to_numeric(belfast_zones['all usual residents'], errors = 'coerce')
#Force to gdf, currently will be a panda series
belfast_zones = gpd.GeoDataFrame(belfast_zones)
#the data from employment has it as count, renamed for clarity
belfast_zones.rename(columns={'count': 'employment_deprivation_count'}, inplace=True)

## Calculate your metrics. Average resident per house etc.
# actual household data has been extracted by spatial join of the randomised poitner dataset. 
#average residents per household
belfast_zones['avg_resi_house'] = (belfast_zones['all usual residents']/belfast_zones['actual_households'])
#average employment deprivation per household
belfast_zones['avg_emp_depri_per_house'] = belfast_zones['employment_deprivation_count']/belfast_zones['actual_households']
#average employment deprivation per person
belfast_zones['avg_emp_depri_per_capita'] = belfast_zones['employment_deprivation_count']/belfast_zones['all usual residents']


